
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>


# Preparação de Dados e Engenharia de Recursos

Nesta demonstração, vamos mergulhar em técnicas como preparar dados de modelagem, incluindo dividir dados, lidar com valores ausentes, codificar recursos categóricos e padronizar recursos. Também discutiremos a remoção de outliers e a coerção de colunas para o tipo de dados correto. Ao final, você terá uma compreensão abrangente da preparação de dados para modelagem e preparação de recursos.

**Objetivos de Aprendizagem:**

Ao final desta demonstração, você será capaz de:

- Coagir colunas para serem do tipo de dados correto com base no tipo de recurso ou variável de destino.
- Identificar e remover outliers dos dados de modelagem.
- Descartar linhas/colunas que contenham valores ausentes.
- Imputar valores ausentes categóricos com o valor de moda.
- Substituir valores ausentes por um valor de substituição especificado.
- Codificar recursos categóricos com um-hot.
- Realizar indexação ordenada como uma alternativa de preparação de recursos categóricos para modelagem de floresta aleatória.
- Aplicar incorporações pré-existentes a recursos categóricos.
- Padronizar recursos em um conjunto de treinamento.
- Dividir dados de modelagem em um dividido treino-teste-retenção como parte de um processo de modelagem.
- Dividir dados de treinamento em conjuntos de validação cruzada como parte de um processo de modelagem.

## Requisitos

Por favor, revise os seguintes requisitos antes de iniciar a lição:

* Para executar este notebook, você precisa usar um dos seguintes tempos de execução do Databricks: **13.3.x-cpu-ml-scala2.12**


## Configuração da Sala de Aula

Antes de iniciar a demonstração, execute o script de configuração da sala de aula fornecido. Este script definirá as variáveis de configuração necessárias para a demonstração. Execute a célula a seguir:

In [0]:
%run ../Includes/Classroom-Setup-01

**Outras Convenções:**

Durante esta demonstração, vamos nos referir ao objeto `DA`. Este objeto, fornecido pela Databricks Academy, contém variáveis como seu nome de usuário, nome do catálogo, nome do esquema, diretório de trabalho e locais dos conjuntos de dados. Execute o bloco de código abaixo para visualizar esses detalhes:

In [0]:
print(f"Username:          {DA.username}")
print(f"Catalog Name:      {DA.catalog_name}")
print(f"Schema Name:       {DA.schema_name}")
print(f"Working Directory: {DA.paths.working_dir}")
print(f"Dataset Location:  {DA.paths.datasets}")


## Limpeza e Imputação de Dados

- Carregue o conjunto de dados do caminho especificado usando Spark e leia-o como um DataFrame.

- Remova quaisquer linhas com valores ausentes do DataFrame usando o método **`dropna()`**.

- Preencha quaisquer valores ausentes restantes no DataFrame com 0 usando o método **`fillna()`**.

- Crie uma visualização temporária chamada **`telco_customer_churn`**

In [0]:
dataset_path = f"{DA.paths.datasets}/telco/telco-customer-churn-noisy.csv"
telco_df = spark.read.csv(dataset_path, header="true", inferSchema="true", multiLine="true", escape='"')

# telco_df.printSchema()
display(telco_df)


### Coagir/Corrigir Tipos de Dados

Mesmo que a maioria dos tipos de dados esteja correta, vamos fazer o seguinte para ter uma pegada de memória melhor do dataframe na memória

* Converter as colunas binárias **`SeniorCitizen`** e **`Churn`** para o tipo booleano.

* Convertendo a coluna **`tenure`** para um inteiro longo usando **`.selectExpr`** e reordenando as colunas.

* Usando **`spark.sql`** para converter as colunas **`Partner`**, **`Dependents`**, **`PhoneService`** e **`PaperlessBilling`** para o tipo booleano, e reordenando as colunas. Em seguida, salvando o dataframe como uma tabela DELTA.

In [0]:
from pyspark.sql.types import BooleanType, ShortType, IntegerType
from pyspark.sql.functions import col, when


binary_columns = ["SeniorCitizen", "Churn"]
telco_customer_churn_df = telco_df
for column in binary_columns:
    telco_customer_churn_df = telco_df.withColumn(column, col(column).cast(BooleanType()))

telco_customer_churn_df.select(*binary_columns).printSchema()

A conversão não funcionou em `SeniorCitizen` provavelmente porque havia alguns valores nulos ou valores que não puderam ser codificados corretamente, podemos forçar a coerção usando um método de filtro simples (supondo que valores ausentes nesta coluna possam ser codificados como `False`)

In [0]:
telco_customer_churn_df = telco_customer_churn_df.withColumn(\
    "SeniorCitizen", when(col("SeniorCitizen")==1, True).otherwise(False))

telco_customer_churn_df.select("SeniorCitizen").printSchema()

In [0]:
# PhoneService & PaperlessBilling para novo booleano usando spark.sql e reordenando colunas
telco_customer_churn_df.createOrReplaceTempView("telco_customer_churn_temp_view")

telco_customer_casted_df = spark.sql("""
    SELECT
        customerID,
        BOOLEAN(Dependents),
        BOOLEAN(Partner),
        BOOLEAN(PhoneService),
        BOOLEAN(PaperlessBilling),
        * 
        EXCEPT (customerID, Dependents, Partner, PhoneService, PaperlessBilling, Churn),
        Churn
    FROM telco_customer_churn_temp_view
""")

telco_customer_casted_df.select("Dependents","Partner","PaperlessBilling", "PhoneService").printSchema()

In [0]:
# Mês de permanência para Long/Integer usando.selectExpr
telco_customer_casted_df = telco_customer_churn_df.selectExpr("* except(tenure)", "cast(tenure as long) tenure")
telco_customer_casted_df.select("tenure").printSchema()

In [0]:
telco_customer_name_full = "telco_customer_full"

# [OPCIONAL] Salvar como tabela DELTA (silver)
telco_customer_full_silver = f"{telco_customer_name_full}_silver"
telco_customer_casted_df.write.mode("overwrite").option("mergeSchema",True).saveAsTable(telco_customer_full_silver)
# print(telco_customer_casted_df)


### Tratamento de Outliers

Veremos como lidar com outliers em uma coluna, identificando e abordando pontos de dados que se encontram muito fora do intervalo típico de valores em um conjunto de dados. Métodos comuns para lidar com outliers incluem removê-los, filtrá-los, transformar os dados ou substituir outliers por valores mais representativos.

Siga estas etapas para lidar com outliers:
* Crie uma nova tabela silver chamada **`telco_customer_full_silver`** acrescentando **`silver`** ao nome da tabela original e, em seguida, acesse-a usando Spark SQL.

* Filtre os outliers da coluna **`TotalCharges`** removendo as linhas em que o valor da coluna excede o valor limite especificado.

Removendo sujos da coluna **`TotalCharges`** removendo linhas onde o valor da coluna excede o valor de corte especificado (por exemplo, valores negativos)

In [0]:
telco_customer_casted_df.select("TotalCharges", "tenure").display()

In [0]:
from pyspark.sql.functions import col


# Remover clientes com TotalCharges negativos
TotalCharges_cutoff = 0

# Use o método.filter e a função SQL col()
telco_no_outliers_df = telco_customer_casted_df.filter(\
    (col("TotalCharges") > TotalCharges_cutoff) | \
    (col("TotalCharges").isNull())) # Manter Nulos

**Removendo outliers do PaymentMethod**
* Identifique os dois grupos com menor ocorrência na coluna **`PaymentMethod`** e calcule a contagem total e a média de **`MonthlyCharges`** para cada grupo.

* Remova os clientes dos grupos identificados de baixa ocorrência na coluna **`PaymentMethod`** para filtrar os outliers.

* Crie um novo dataframe **`telco_filtered_df`** contendo os dados filtrados.

* Compare a contagem de registros antes e depois dividindo a contagem de **`telco_casted_full_df`** e **`telco_no_outliers_df`** dataframe removendo outliers e, em seguida, materialize o dataframe resultante como uma nova tabela.

In [0]:
from pyspark.sql.functions import col, count, avg

# Identificar 2 grupos de ocorrências mais baixas
group_var = "PaymentMethod"
stats_df = telco_no_outliers_df.groupBy(group_var) \
                      .agg(count("*").alias("Total"),\
                           avg("MonthlyCharges").alias("MonthlyCharges")) \
                      .orderBy(col("Total").desc())

# Exibir
display(stats_df)

In [0]:
# Reúna os 2 grupos mais baixos supondo que o limiar de contagem é inferior a 20% do conjunto de dados completo e encargos mensais <20$
N = telco_no_outliers_df.count()  # contagem total
lower_groups = [elem[group_var] for elem in stats_df.head(2) if elem['Total']/N < 0.2 and elem['MonthlyCharges'] < 20]
print(f"Removendo grupos: {', '.join(lower_groups)}")

In [0]:
# Filtrar/remover listagens destes grupos de baixa ocorrência mantendo ocorrências nulas
telco_no_outliers_df = telco_no_outliers_df.filter( \
    ~col(group_var).isin(lower_groups) | \
    col(group_var).isNull())

In [0]:
# Contar/Comparar conjuntos de dados antes/depois de remover outliers
print(f"Contagem - Antes: {telco_customer_casted_df.count()} / Depois: {telco_no_outliers_df.count()}")

In [0]:
# Materialize/Snap table [OPTIONAL/for instructor only]
telco_no_outliers_df.write.mode("overwrite").saveAsTable(telco_customer_full_silver)


### Tratamento de Valores Ausentes

Para lidar com valores ausentes no conjunto de dados, é necessário identificar colunas com altas porcentagens de dados ausentes e descartar essas colunas. Em seguida, ele remove linhas com valores ausentes. Colunas numéricas são preenchidas com 0, e colunas de string são preenchidas com 'N/A'. No geral, o código demonstra uma abordagem abrangente para lidar com valores ausentes no conjunto de dados.


#### Eliminar Colunas

* Crie um DataFrame chamado **`missing_df`** para contar os valores ausentes por coluna no conjunto de dados **`telco_no_outliers_df`**.

* O DataFrame **`missing_df`** é então transposto para uma leitura mais fácil usando a função TransposeDF, o que permite uma análise mais fácil dos valores ausentes.

In [0]:
from pyspark.sql.functions import col, when, count, concat_ws, collect_list # isnan


def calculate_missing(input_df, show=True):
  """
  Função auxiliar para calcular e exibir dados ausentes
  """

  # Primeiro obter contagem de valores ausentes por coluna para obter um DF de linha singleton
  missing_df_ = input_df.select([count(when(col(c).contains('None') | \
                                                  col(c).contains('NULL') | \
                                                  (col(c) == '' ) | \
                                                  col(c).isNull(), c)).alias(c) \
                                                  for c in input_df.columns
                                            ])

  # Transpor para melhor legibilidade
  def TransposeDF(df, columns, pivotCol):
    """Função auxiliar para transpor dataframe do spark"""
    columnsValue = list(map(lambda x: str("'") + str(x) + str("',")  + str(x), columns))
    stackCols = ','.join(x for x in columnsValue)
    df_1 = df.selectExpr(pivotCol, "stack(" + str(len(columns)) + "," + stackCols + ")")\
            .select(pivotCol, "col0", "col1")
    final_df = df_1.groupBy(col("col0")).pivot(pivotCol).agg(concat_ws("", collect_list(col("col1"))))\
                  .withColumnRenamed("col0", pivotCol)
    return final_df

  missing_df_out_T = TransposeDF(
    spark.createDataFrame([{"Column":"Número de Valores Ausentes"}]).join(missing_df_),
    missing_df_.columns,
    "Column").withColumn("Número de Valores Ausentes", col("Número de Valores Ausentes").cast("long"))

  if show:
    display(missing_df_out_T.orderBy("Número de Valores Ausentes", ascending=False))

  return missing_df_out_T

missing_df = calculate_missing(telco_no_outliers_df)

**Remover colunas com mais de x% de linhas faltantes**

Colunas com mais de 60% de dados faltantes são identificadas e armazenadas na lista **`to_drop_missing`**, e essas colunas são posteriormente removidas do conjunto de dados **`telco_no_outliers_df`**.

In [0]:
per_thresh = 0.6 # Drop if column has more than 80% missing data

N = telco_no_outliers_df.count() # total count
to_drop_missing = [x.asDict()['Column'] for x in missing_df.select("Column").where(col("Number of Missing Values") / N >= per_thresh).collect()]

In [0]:
print(f"Eliminando colunas {to_drop_missing} para mais de {per_thresh * 100}% de dados ausentes")
telco_no_missing_df = telco_no_outliers_df.drop(*to_drop_missing)
# display(telco_no_missing_df)

**Remover linhas que contêm números específicos de colunas/campos ausentes**

Linhas com mais de 1/4 das colunas com valores ausentes são descartadas usando **`na.drop()`** e os valores ausentes restantes nas colunas numéricas são imputados com 0, enquanto os valores ausentes nas colunas de string são imputados com 'N/A'.

In [0]:
n_cols = len(telco_no_missing_df.columns)
telco_no_missing_df = telco_no_missing_df.na.drop(how='any', thresh=round(n_cols/4)) # Elimine as linhas em que pelo menos metade dos valores estão faltando, how='all' também pode ser usado

In [0]:
# Contar/Comparar conjuntos de dados antes/depois de remover ausentes
print(f"Contagem - Antes: {telco_no_outliers_df.count()} / Depois: {telco_no_missing_df.count()}")


#### Imputar Dados Ausentes

Substitua valores ausentes por um valor de substituição especificado.

* As listas **`num_cols`** e **`string_cols`** são criadas para identificar colunas numéricas e de string no conjunto de dados, respectivamente.

* Finalmente, valores ausentes nas colunas numéricas e de string são imputados com valores apropriados usando **`na.fill()`**, resultando no conjunto de dados **`telco_imputed_df`**.

**Substituir numérico faltante com constante/0**

OBSERVAÇÃO: não se aplica ao caso deste conjunto de dados

In [0]:
from pyspark.sql.types import DoubleType, IntegerType


# Obtenha uma lista de colunas numéricas
num_cols = [c.name for c in telco_no_missing_df.schema.fields if (c.dataType == DoubleType() or c.dataType == IntegerType())]

In [0]:
# Imputar
# telco_imputed_df = telco_no_missing_df.na.fill(value=0, subset=num_cols)

**Substitua booleano ausente com `False`**

In [0]:
from pyspark.sql.types import BooleanType


# Obtenha uma lista de colunas booleanas
bool_cols = [c.name for c in telco_no_missing_df.schema.fields if (c.dataType == BooleanType())]

# Imputar
telco_imputed_df = telco_no_missing_df.na.fill(value=False, subset=bool_cols)

**Substitua a string missing com `No`**

Todas as colunas de string, exceto `gender`, `Contract` e `PaymentMethod`

In [0]:
from pyspark.sql.types import StringType


# Obter lista de colunas de string
to_exclude = ["customerID", "gender", "Contract", "PaymentMethod"]
string_cols = [c.name for c in telco_no_missing_df.drop(*to_exclude).schema.fields if c.dataType == StringType()]

# Imputar
telco_imputed_df = telco_imputed_df.na.fill(value='No', subset=string_cols)

In [0]:
# Comparar estatísticas ausentes novamente
calculate_missing(telco_imputed_df)

In [0]:
telco_imputed_df.write.mode("overwrite").saveAsTable(f"{DA.catalog_name}.{DA.schema_name}.telco_imputed_silver")


## Codificação de Recursos Categóricos

Nesta seção, nós iremos codificar um-hot os recursos categóricos/string usando o avaliador `OneHotEncoder` da Spark MLlib.

Se você não está familiarizado com a codificação one-hot, há uma descrição abaixo. Se você já está familiarizado, pode pular para a seção **Codificação one-hot no Spark MLlib** em direção ao fundo da célula.

#### Recursos categóricos no aprendizado de máquina

Muitos algoritmos de aprendizado de máquina não são capazes de aceitar recursos categóricos como entradas. Como resultado, cientistas de dados e engenheiros de aprendizado de máquina precisam determinar como lidar com eles.

Uma solução fácil seria remover os recursos categóricos do conjunto de recursos. Embora isso seja rápido, **você está removendo informações potencialmente preditivas** &mdash; portanto, geralmente não é a melhor estratégia.

Outras opções incluem maneiras de representar recursos categóricos como recursos numéricos. Algumas opções comuns são:

1. **Codificação one-hot**: criar variáveis dummy/binárias para cada categoria
2. **Codificação de rótulo/alvo**: substituir cada valor de categoria com um valor que representa a variável de destino (por exemplo, substituir um valor de categoria específico com a média da variável de destino para linhas com esse valor de categoria)
3. **Embutimentos**: usar/criar uma representação vetorial de palavras significativas em cada valor de categoria

Cada uma dessas opções pode ser realmente útil em diferentes cenários. Vamos nos concentrar na codificação one-hot aqui.

#### Noções básicas de codificação one-hot

A codificação one-hot cria um recurso binário/dummy para cada categoria em cada recurso categórico.

No exemplo abaixo, o recurso **Animal** é dividido em três recursos binários &mdash; um para cada valor em **Animal**. O valor de cada recurso binário é igual a 1 se seu respectivo valor de categoria estiver presente em **Animal** para cada linha. Se seu valor de categoria não estiver presente na linha, o valor do recurso binário será 0.

![Imagem de codificação one-hot](https://s3-us-west-2.amazonaws.com/files.training.databricks.com/images/mlewd/Scaling-Machine-Learning-Pipelines/one-hot-encoding.png)

#### Codificação one-hot no Spark MLlib

Mesmo que você entenda a codificação one-hot, é importante aprender como executá-la usando o Spark MLlib.

Para codificar one-hot recursos categóricos no Spark MLlib, usaremos duas classes: [a classe **`StringIndexer`**](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.StringIndexer.html#pyspark.ml.feature.StringIndexer) e [a classe **`OneHotEncoder`**](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.OneHotEncoder.html#pyspark.ml.feature.OneHotEncoder).

* A classe `StringIndexer` indexa colunas do tipo string para um índice numérico. Cada valor exclusivo na coluna do tipo string é mapeado para um inteiro exclusivo.
* A classe `OneHotEncoder` aceita colunas indexadas e as converte em um recurso do tipo vetor codificado em one-hot.

#### Aplicando o fluxo de trabalho `StringIndexer` -> `OneHotEncoder` -> `VectorAssembler`

Primeiro, precisaremos indexar os recursos categóricos do DataFrame. `StringIndexer` recebe alguns argumentos:

1. Uma lista de colunas categóricas para indexar.
2. Uma lista de nomes para as colunas indexadas que estão sendo criadas.
3. Instruções sobre como lidar com novas categorias ao transformar dados.

Como `StringIndexer` precisa aprender quais categorias estão presentes antes de indexar, é um **estimador** &mdash; lembre-se de que isso significa que precisamos chamar seu método `fit`. Seu resultado pode então ser usado para transformar nossos dados.

In [0]:
sample_df = telco_imputed_df.select("Contract").distinct()
sample_df.show()

In [0]:
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col


# StringIndexer
string_cols = ["Contract"]
index_cols = [column + "_index" for column in string_cols]

string_indexer = StringIndexer(inputCols=string_cols, outputCols=index_cols, handleInvalid="skip")
string_indexer_model = string_indexer.fit(sample_df)
indexed_df = string_indexer_model.transform(sample_df)

indexed_df.show()

Uma vez que nossos dados tenham sido indexados, estamos prontos para usar o estimador `OneHotEncoder`.

<img src="https://files.training.databricks.com/images/icon_hint_24.png"/>&nbsp;**Dica:** Veja a [documentação do `OneHotEncoder`](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.OneHotEncoder.html#pyspark.ml.feature.OneHotEncoder) e nossos fluxos de trabalho anteriores do Spark MLlib que usam estimadores para obter orientação.

In [0]:
from pyspark.ml.feature import OneHotEncoder


# Crie uma lista de nomes de recursos codificados em um-hot
ohe_cols = [column + "_ohe" for column in string_cols]

# Instancie o OneHotEncoder com as listas de colunas
ohe = OneHotEncoder(inputCols=index_cols, outputCols=ohe_cols, handleInvalid="keep")

# Ajuste o OneHotEncoder nos dados indexados
ohe_model = ohe.fit(indexed_df)

# Transforme indexed_df usando o ohe_model
ohe_df = ohe_model.transform(indexed_df)
ohe_df.show()

In [0]:
from pyspark.ml.feature import VectorAssembler


selected_ohe_cols = ["Contract_ohe"]

# Use VectorAssembler para montar as colunas codificadas one-hot selecionadas em um vetor denso
assembler = VectorAssembler(inputCols=selected_ohe_cols, outputCol="features")
result_df_dense = assembler.transform(ohe_df)

# Selecione colunas relevantes para exibição
result_df_display = result_df_dense.select("Contract", "features")

result_df_display.show(truncate=False)


### Aplique incorporações pré-existentes a recursos categóricos/disseminados

Vamos agrupar **`tenure`** para converter os dados discretos em formato de categorias/bins para uma análise e modelagem posteriores.

In [0]:
column_to_bin = "tenure"
display(telco_imputed_df.select(column_to_bin))

In [0]:
from pyspark.ml.feature import Bucketizer
from pyspark.sql.functions import col


# Especificar faixas de bin e coluna para binarizar
bucketizer = Bucketizer(
    splits=[0, 24, 48, float('Inf')],
    inputCol=column_to_bin,
    outputCol=f"{column_to_bin}_bins"
)

# Aplicar o bucketizer ao DataFrame
bins_df = bucketizer.transform(telco_imputed_df.select(column_to_bin))

# Reformule os números bin para inteiros
bins_df = bins_df.withColumn(f"{column_to_bin}_bins", col(f"{column_to_bin}_bins").cast("integer"))

# Mostrar o resultado
display(bins_df)

Mapear de volta para os escores de embedding legíveis por humanos

In [0]:
bins_embedded_df = (
  bins_df.withColumn(f"{column_to_bin}_embedded", col(f"{column_to_bin}_bins").cast(StringType()))
         .replace(to_replace = 
                  {
                    "0":"<2y",
                    "1":"2-4y",
                    "2":">4y"
                  },
                  subset=[f"{column_to_bin}_embedded"])
)
display(bins_embedded_df)


### Indexação Ordenada

Realize indexação ordenada como uma preparação alternativa de recursos categóricos para modelagem de floresta aleatória.

Algumas categorias são, na verdade, `ordinais` e, portanto, podem exigir codificação adicional/manual

In [0]:
ordinal_cat = "Contract"
telco_imputed_df.select(ordinal_cat).distinct().show(truncate=False)

In [0]:
# Definir mapa/dicionário de Ordinal (categoria:índice)
ordered_list = [
    "Month-to-month",
    "One year",
    "Two year"
]

ordinal_dict = {category: f"{index+1}" for index, category in enumerate(ordered_list)}
display(ordinal_dict)

In [0]:
# Crie uma nova coluna com indexação ordenada
from pyspark.sql.functions import expr


ordinal_df = (
    telco_imputed_df
   .withColumn(f"{ordinal_cat}_ord", col(ordinal_cat)) # Duplicar
   .replace(to_replace=ordinal_dict, subset=[f"{ordinal_cat}_ord"]) # Mapear
   .withColumn(f"{ordinal_cat}_ord", col(f"{ordinal_cat}_ord").cast('int')) # Converter para inteiro
)

display(ordinal_df.select(ordinal_cat, f"{ordinal_cat}_ord"))


## Dividindo Dados (para Validação Cruzada)

Divida os dados de modelagem em um conjunto de treino, teste e validação como parte de um processo de modelagem

Nesta seção, realizaremos o fluxo de trabalho recomendado para uma divisão treino-teste usando a API DataFrame do Spark.

Lembre-se de que, devido a fatores como configurações de cluster em constante mudança e particionamento de dados, pode ser difícil garantir uma divisão treino-teste reproduzível. Como resultado, recomendamos:

1. Divida os dados usando a **mesma semente aleatória**
2. Escreva os DataFrames de treino e teste

<img src="https://files.training.databricks.com/images/icon_hint_24.png"/>&nbsp;**Dica:** Confira a [**documentação `randomSplit`**](https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.DataFrame.randomSplit.html).

In [0]:
# Dividir com 80 por cento dos dados em train_df e 20 por cento dos dados em test_df
train_df, test_df = telco_imputed_df.randomSplit([.8,.2], seed=42)

In [0]:
# Materialize (OPCIONAL)
train_df.write.mode("overwrite").option("overwriteSchema", True).saveAsTable(f"{DA.catalog_name}.{DA.schema_name}.telco_customers_train")
test_df.write.mode("overwrite").option("overwriteSchema", True).saveAsTable(f"{DA.catalog_name}.{DA.schema_name}.telco_customers_baseline")


### Padronizar Recursos em um Conjunto de Treinamento

Para fins de exemplo, escolheremos uma coluna sem dados ausentes (por exemplo, `MonthlyCharges`)

In [0]:
from pyspark.ml.feature import StandardScaler, RobustScaler, VectorAssembler


num_cols_to_scale = ["MonthlyCharges"] # colunas numéricas
assembler = VectorAssembler().setInputCols(num_cols_to_scale).setOutputCol("numerical_assembled")
train_assembled_df = assembler.transform(train_df.select(*num_cols_to_scale))
test_assembled_df = assembler.transform(test_df.select(*num_cols_to_scale))

# Define o escalonador e ajusta-o no conjunto de treinamento
scaler = RobustScaler(inputCol="numerical_assembled", outputCol="numerical_scaled")
scaler_fitted = scaler.fit(train_assembled_df)


# Aplica-se a ambos os conjuntos de treinamento e teste
train_scaled_df = scaler_fitted.transform(train_assembled_df)
test_scaled_df = scaler_fitted.transform(test_assembled_df)

In [0]:
print("Peek at Training set")
train_scaled_df.show(5)

In [0]:
print("Peek at Test set")
test_scaled_df.show(5)

### Imputar valores categoricos ausentes com o valor de moda usando sparkml

Como lidar com dados ausentes apenas no momento do treinamento e incorporá-los como parte do pipeline de inferência para evitar vazamentos de dados e garantir que a observação com dados ausentes seja usada para treinamento.

In [0]:
categorical_cols_to_impute = ["PaymentMethod"] # string_cols

Indexar categoricals primeiro, pois `Imputer` não lida com categoricals diretamente

In [0]:
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col


# Indexar colunas categóricas usando StringIndexer
cat_index_cols = [column + "_index" for column in categorical_cols_to_impute]
cat_indexer = StringIndexer(
    inputCols=categorical_cols_to_impute,
    outputCols=cat_index_cols,
    handleInvalid="keep"
)

# Treinar no conjunto de treinamento
cat_indexer_model = cat_indexer.fit(train_df.select(categorical_cols_to_impute))

In [0]:
# Transformar ambos os conjuntos de treino e teste usando o modelo ajustado StringIndexer
cat_indexed_train_df = cat_indexer_model.transform(train_df.select(*categorical_cols_to_impute))
cat_indexed_test_df = cat_indexer_model.transform(test_df.select(*categorical_cols_to_impute))
# display(cat_indexed_train_df)

In [0]:
cat_indexed_train_df.display()

O `StringIndexer` criará um novo rótulo (_por exemplo_, `4`) para ausente ao definir a flag `handleInvalid` como `keep`, por isso é importante manter o controle/reverter os valores de índice de volta para `null` se quisermos imputá-los, caso contrário `null` será tratado como sua própria/categoria separada automaticamente.

Como alternativa para imputar categorias/strings, podemos usar o método `.fillna()` fornecendo o valor `mode` manualmente (conforme descrito acima).

In [0]:
# Revert indexes to `null` for missing categories
for c in categorical_cols_to_impute:
    cat_indexed_train_df = cat_indexed_train_df.withColumn(f"{c}_index", when(col(c).isNull(), None).otherwise(col(f"{c}_index")))
    cat_indexed_test_df = cat_indexed_test_df.withColumn(f"{c}_index", when(col(c).isNull(), None).otherwise(col(f"{c}_index")))

In [0]:
cat_indexed_train_df.display()

Ajuste o imputer nos categoricals indexados

In [0]:
from pyspark.ml.feature import Imputer


# Definir modo de imputação
output_cat_index_cols_imputed = [col+'_imputed' for col in cat_index_cols]
mode_imputer = Imputer(
  inputCols=cat_index_cols,
  outputCols=output_cat_index_cols_imputed,
  strategy="mode"
  )

# Ajustar em training_df
mode_imputer_fitted = mode_imputer.fit(cat_indexed_train_df)

In [0]:
# Transforme conjuntos de treinamento e teste
cat_indexed_train_imputed_df = mode_imputer_fitted.transform(cat_indexed_train_df)
cat_indexed_test_imputed_df  = mode_imputer_fitted.transform(cat_indexed_test_df)

In [0]:
# Vislumbre o conjunto de testes
display(cat_indexed_test_imputed_df)


## Limpar Sala de Aula

Execute a célula a seguir para remover os ativos específicos de lições criados durante esta lição.

In [0]:
DA.cleanup()


## Conclusão

Esta demonstração forneceu uma compreensão abrangente da preparação de dados para modelagem e preparação de recursos, equipando-o com o conhecimento e habilidades para preparar seus dados de forma eficaz para modelagem e análise. Vimos de maneira contínua como corrigir o tipo de dados, identificar e remover outliers, lidar com valores ausentes por meio de imputação ou substituição, codificar recursos categóricos e padronizar recursos.

&copy; 2024 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>